In [ ]:
# This script is designed to do somee sizing tests for Enterprise Search using Elastic Cloud
# It will use the Elastic Cloud CLI. Please asetup as per the docs
# https://www.elastic.co/guide/en/ecctl/current/index.html
# brew tap elastic/tap
# brew install elastic/tap/ecctl
# ecctl init 
# Need to add the App Search specific python client
# python -m pip install elastic-app-search

### Provision a cloud cluster
# In this rolder run:
# ecctl deployment create --output json -f ./deployment-config.json > ./cluster-info.json

In [1]:

engine_name = "sizing"
cluster_info_path= "./cluster-info.json"


In [5]:
import os
import json
import time
import requests
import subprocess
from requests.auth import HTTPBasicAuth
from urllib.parse import urljoin
from elastic_app_search import Client

In [11]:
# So there is currently a bug in ecctl which should be fixed soon.
# until then please run the command directly in the terminal to create the file.
# when the issue is fixed we can reuse this code
#
# cctl deployment create --track -f ./deployment-config.json > cluster-info.json

#print("Checking if the cluster is already created")
#if not(os.path.exists(cluster_info_path) and os.path.getsize(cluster_info_path)>0):
#    print("No cluster info file so lets create a new cluster in cloud")
#    info_file=open(cluster_info_path,"w+")
#    out = subprocess.run(args=['ecctl', 'deployment', 'create', '-f ./deployment-config.json'], 
#           stdout=info_file, 
#           stderr=subprocess.STDOUT)
#    stdout,stderr = out.communicate()
#    print(stdout,stderr)
        

Checking if the cluster is already created


In [47]:

print("Reading cluster info file: ",cluster_info_path)
with open(cluster_info_path) as json_file:
    cluster_info = json.load(json_file)
    deployment_name = cluster_info['name']
    print("Cluster name: ",deployment_name)
    deployment_id = cluster_info['id']
    print("Cluster ID: ",deployment_id)
    cloud_id = cluster_info['resources'][0]['cloud_id']
    print("CloudID: ",cloud_id)
    elastic_password = cluster_info['resources'][0]['credentials']['password']
    print("elastiic password: ",elastic_password)
    cluster_exists = True

Reading cluster info file:  ./cluster-info.json
Cluster name:  ent-search-sizing
Cluster ID:  ec9e96c0b04d92dab0a7be7127ec967e
CloudID:  ent-search-sizing:ZXVyb3BlLXdlc3QxLmdjcC5jbG91ZC5lcy5pbyQ1YjVjMGQ0ZWZhNTU0MDFlOTgxMjFjZTdiODJjMjc3NSQ0Y2NjNzJjNDZhOTk0NTg4OTAzMmEwZmE3NjFjZjNlNw==
elastiic password:  gn2cADroer7rmbTqsFSCbLHF


In [49]:
out = subprocess.run(['ecctl', 'deployment', 'show', deployment_id], 
           stdout=subprocess.PIPE, 
           stderr=subprocess.STDOUT)
cluster_details = json.loads(out.stdout)

In [58]:
# Now we can extract any details we need
as_endpoint = cluster_details['resources']['enterprise_search'][0]['info']['metadata']['endpoint']
print('App Search endpoint: ',as_endpoint)
es_endpoint = cluster_details['resources']['elasticsearch'][0]['info']['metadata']['endpoint']
print('Elasticsearch endpoint: ',es_endpoint)



App Search endpoint:  6eadb1cc7a794a1e86e326be74e48f0b.ent-search.europe-west1.gcp.cloud.es.io
Elasticsearch endpoint:  5b5c0d4efa55401e98121ce7b82c2775.europe-west1.gcp.cloud.es.io


In [ ]:
response =requests.get('https://api.github.com/user', auth=('elastic', elastic_password))
print(response)

In [72]:
# The App Search Client needs an API key so we will need to call the API directly using basic auth to generate a key

# The only way to get this atm is through an undocumented API
create_key_url = urljoin("https://"+as_endpoint, '/as/credentials/collection?page[current]=1')
print(create_key_url)
response =requests.get(create_key_url, auth=('elastic', elastic_password))
print(response)
api_key_info = response.json()
for key in api_key_info['results']:
    if key['type'] == "private":
        private_key = key['key']
        print("Private key: ",private_key)
    elif key['type'] == "search":
        search_key = key['key']
        print("Search key: ",search_key)
#as_client = Client(

6eadb1cc7a794a1e86e326be74e48f0b.ent-search.europe-west1.gcp.cloud.es.io
https://6eadb1cc7a794a1e86e326be74e48f0b.ent-search.europe-west1.gcp.cloud.es.io/as/credentials/collection?page[current]=1
<Response [200]>
Private key:  private-8pytm5nms91djqdnb2s54vun
Search key:  search-q6vrf4f7ipdg8f5zck66uu67


In [73]:

base_endpoint = urljoin
as_client = Client(
    base_endpoint=as_endpoint,
    api_key=private_key,
    use_https=True
)

In [74]:
as_client.create_engine(engine_name, 'en')

NonExistentRecord: 

In [76]:
as_client.destroy_engine('favorite-videos')

NonExistentRecord: 